### Find out top 50 products

In [12]:
import pandas as pd

# Load the original CSV file
df = pd.read_csv("Ecom_data.csv")

# Ensure QUANTITY is numeric, coercing errors to NaN
df['QUANTITY'] = pd.to_numeric(df['QUANTITY'], errors='coerce')

# Fill NaN values in QUANTITY with 0 (or handle as needed)
df['QUANTITY'] = df['QUANTITY'].fillna(0)

# First aggregation: Sum quantities based on STYLE NO. and COLOR
aggregated_df = df.groupby(['STYLE NO.', 'COLOR', "SIZE"], as_index=False).agg({
    'BRAND NAME': 'first',
    'QUANTITY': 'sum'
})

# Second aggregation: Sum quantities based on STYLE NO. to get total quantity for each style
style_quantity_df = aggregated_df.groupby('STYLE NO.', as_index=False).agg({
    'QUANTITY': 'sum'
})

# Sort by QUANTITY in descending order and select the top 50 styles
top_50_styles = style_quantity_df.sort_values(
    by='QUANTITY', ascending=False).head(50)['STYLE NO.']

# Filter the original aggregated DataFrame to include only rows corresponding to the top 50 styles
top_50_df = aggregated_df[aggregated_df['STYLE NO.'].isin(top_50_styles)]

# Write the result to a new CSV file
top_50_df.to_csv('csv_data/top_50_products.csv', index=False)

print('The top 50 unique styles based on quantity have been written to top_50_products.csv')

The top 50 unique styles based on quantity have been written to top_50_products.csv


KeyError: 'Handle'

### Merge TOP 50 CSV and JSON of products to new JSON file with all the variant combinations of SIZE and COLOR

In [ ]:
import json
import pandas as pd

# Load the JSON file
with open('our_products_new.json', 'r') as json_file:
    json_data = json.load(json_file)

# Load the CSV file into a pandas DataFrame
csv_data = pd.read_csv('csv_data/top_50_products.csv')

count = 0
# Iterate over the CSV rows and update the JSON data
for _, row in csv_data.iterrows():
    style_no = row['STYLE NO.']
    color = row['COLOR']
    size = row['SIZE']
    quantity = row['QUANTITY']

    # Find the matching dict in the JSON data
    for item in json_data:
        # print(item.get('code'), style_no)

        if item.get('code') == style_no:
            if 'variants' not in item:
                item['variants'] = []
            # Add the color, size, and quantity as a new variant
            item['variants'].append({
                'color': color,
                'size': size,
                'quantity': quantity
            })

# # Save the updated list of dicts to a new JSON file
with open('updated_data.json', 'w') as output_file:
    json.dump(json_data, output_file, indent=4)

print('Updated data has been written to updated_data.json')


### Generating Shopify CSV for PRODUCTS


In [ ]:
import json
import csv

# Define the required headers for the CSV
headers = ["Handle", "Title", "Body (HTML)", "Description", "Vendor", "Type", "Option1 Name", "Option1 Value", "Option2 Name", "Option2 Value", "Variant SKU", "Variant Inventory Qty",
           "Variant Price", "Variant Inventory Policy", "Variant Requires Shipping", "Variant Taxable", "Image Src", "Variant Inventory Tracker"]

# Load the JSON file
with open('updated_data.json', 'r') as json_file:
    json_data = json.load(json_file)

# Open a new CSV file for writing
with open('DEMO_shopify_data_1.csv', 'w', newline='') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=headers)

    # Write the header row
    writer.writeheader()

    # Write the JSON data to the CSV file
    for index, item in enumerate(json_data):
        # Create a dictionary for the CSV row
        img = item["additional_images_url"][0]
        if "variants" in item.keys():
            for variant in item['variants']:
                csv_row = {
                    "Handle": item.get("code").lower(),
                    "Title": item.get("code") + " " + item.get("name"),
                    "Body (HTML)": item.get("description"),
                    "Description": item.get("description"),
                    "Vendor": "Milano Formals",
                    "Type": "Dress",
                    "Option1 Name": "Size",
                    "Option1 Value": variant['size'],
                    "Option2 Name": "Color",
                    "Option2 Value": variant['color'],
                    "Variant SKU": item.get("code") + "_" + variant['color'] + "_" + variant['size'],
                    "Variant Inventory Tracker": "shopify",
                    "Variant Inventory Qty": variant['quantity'],
                    "Variant Price": "$200",
                    "Variant Inventory Policy":  "deny",
                    "Variant Requires Shipping": "TRUE",
                    "Variant Taxable": "TRUE",
                    "Image Src": img,
                }
                # Write the row to the CSV file
                writer.writerow(csv_row)

print('JSON data has been converted to CSV and written to output.csv')


In [5]:
import pandas as pd

# Load the CSV file into a pandas DataFrame
# csv_data = pd.read_csv('csv_data/top_50_products_new.csv')
# code = "STYLE NO."

# csv_data = pd.read_csv('csv_data/shopify_data_new.csv')
csv_data = pd.read_csv('DEMO_shopify_data_1.csv')
code = "Handle"

unique_style_no_count = csv_data[code].nunique()

# Print the result
print(f"Number of unique 'STYLE NO.': {unique_style_no_count}")

# List all unique values in the column
unique_values = csv_data[code].unique()
print(f"Unique values in Handle:")
print(unique_values)

Number of unique 'STYLE NO.': 30
Unique values in Handle:
['b8865' 'b8998' 'e1168' 'e1260' 'e1264' 'e1270' 'e1285' 'e1295' 'e1340'
 'e1379' 'e1420' 'e1421' 'e1463' 'e1502' 'e1520' 'e1531' 'e1544' 'e1581'
 'e1582' 'e1657' 'e1663' 'e1696' 'e1729' 'e1730' 'e1796' 'e1799' 'f1116'
 'f570' 'e2107' 'e1013']


### Checking missing products

In [7]:
import json
import pandas as pd

# Load the JSON file
with open('our_products_new.json', 'r') as json_file:
    json_data = json.load(json_file)

# Extract the 'code' values from the JSON data
json_codes = {item['code'] for item in json_data}

# Load the CSV file into a pandas DataFrame
csv_data = pd.read_csv('csv_data/top_50_products.csv')

# Extract the 'STYLE NO.' values from the CSV data
csv_codes = set(csv_data['STYLE NO.'])

# Find missing items in JSON that are not in CSV
missing_in_json = json_codes - csv_codes

# Find missing items in CSV that are not in JSON
missing_in_csv = csv_codes - json_codes

print("Items in JSON but not in CSV:")
print(missing_in_json)

print("Items in CSV but not in JSON:")
print(missing_in_csv)


Items in JSON but not in CSV:
{'E2158', 'E2281', 'E2326', 'E2276', 'E2244', 'E2254', 'E2296', 'E2324', 'E1852', 'E1573', 'E2043', 'E2333', 'E2329', 'E2335', 'E2081', 'E2448', 'E2317', 'E2171', 'E2255', 'E1827', 'E1940', 'E2320', 'E2460', 'E2050', 'E2428', 'E2150', 'E2434', 'E2269', 'E1988', 'E2282', 'E1715', 'E2287', 'E2426', 'E2318', 'E2262', 'E1259', 'E2280', 'E2210', 'E2046', 'E2243', 'E1774', 'E2227', 'E2049', 'E2060', 'E2437', 'E2360', 'E2385', 'E2331', 'E1637', 'E2328', 'E2233', 'E2279', 'E2395', 'E2088', 'E2410', 'E2085', 'E2080', 'E1822', 'E2256', 'E2086', 'E2160', 'E2032', 'E2028', 'E1838', 'E2241', 'E1546', 'E1028', 'E2053', 'E2291', 'E2213', 'E2245', 'E2042', 'E2078', 'E2259', 'E2389', 'E2114', 'E2366', 'E2025', 'E2031', 'E2356', 'E2257', 'E2251', 'E1986', 'E2036', 'E2334', 'E2394', 'E2289', 'E2449', 'E2436', 'E1824', 'E2346', 'E2433', 'E2405', 'E2252', 'E2161', 'E2344', 'E2336', 'E2292', 'E1974', 'E2225', 'E2267', 'E1967', 'E2079', 'E2229', 'E2322', 'E2253', 'E2327', 'E2340